In [1]:
import pandas as pd

# Load the CSV file
csv_file = 'rawdata/etsb.csv'
df = pd.read_csv(csv_file)



# Print the first few rows
print("Head of the file:")
print(df.head())

# Print the last few rows
print("\nTail of the file:")
print(df.tail())


Head of the file:
   Week Number  Time of Week  Solution Status  Number of Channels  PRN Number  \
0          260      200603.0                0                  12          13   
1          260      200603.0                0                  12          24   
2          260      200603.0                0                  12          16   
3          260      200603.0                0                  12          17   
4          260      200603.0                0                  12          22   

   Channel Tracking Status      Doppler       C/N0  Residual  Locktime  \
0                    20481  3250.391647   0.000000  0.000000      0.00   
1                    24084   461.078240  41.789088  1.398459    562.97   
2                    20513   500.140753   0.000000  0.000000      0.00   
3                    24116 -1447.297381  37.501903 -3.403472    520.18   
4                    24132  -460.359560  43.856835  0.230845    567.75   

    Pseudorange  Rejection Code  
0  0.000000e+00 

In [2]:
# Drop all columns except the specified ones
columns_to_keep = ['Week Number', 'Time of Week', 'PRN Number', 'C/N0']
df = df[columns_to_keep]

# Print the first few rows
print("Head of the file:")
print(df.head())

# Print the last few rows
print("\nTail of the file:")
print(df.tail())

Head of the file:
   Week Number  Time of Week  PRN Number       C/N0
0          260      200603.0          13   0.000000
1          260      200603.0          24  41.789088
2          260      200603.0          16   0.000000
3          260      200603.0          17  37.501903
4          260      200603.0          22  43.856835

Tail of the file:
         Week Number  Time of Week  PRN Number       C/N0
1036999          260      287019.0          11  45.568371
1037000          260      287019.0           5  35.359862
1037001          260      287019.0          24  41.120052
1037002          260      287019.0          20  40.645080
1037003          260      287019.0          22  43.691339


In [3]:
from datetime import datetime, timedelta

# Define the GPS epoch
GPS_EPOCH = datetime(1980, 1, 6)

def gps_to_utc(week, seconds):
    # Calculate the GPS time
    gps_time = GPS_EPOCH + timedelta(weeks=week, seconds=seconds)
    return gps_time

# Apply the conversion to the DataFrame
df['UTC Time'] = df.apply(lambda row: gps_to_utc(row['Week Number'], row['Time of Week']), axis=1)

# Convert 'UTC Time' to a 24-hour time format
df['24-hour Time'] = df['UTC Time'].dt.strftime('%H:%M:%S')

# Print the first few rows to verify
print("Head of the file with 24-hour time format:")
print(df.head())

Head of the file with 24-hour time format:
   Week Number  Time of Week  PRN Number       C/N0            UTC Time  \
0          260      200603.0          13   0.000000 1985-01-01 07:43:23   
1          260      200603.0          24  41.789088 1985-01-01 07:43:23   
2          260      200603.0          16   0.000000 1985-01-01 07:43:23   
3          260      200603.0          17  37.501903 1985-01-01 07:43:23   
4          260      200603.0          22  43.856835 1985-01-01 07:43:23   

  24-hour Time  
0     07:43:23  
1     07:43:23  
2     07:43:23  
3     07:43:23  
4     07:43:23  


In [4]:
import plotly.io as pio
pio.renderers.default = "iframe"
# Ask user to input a specific PRN
import plotly.express as px
import plotly.graph_objects as go
# Plot d/u values against 24-hour time for all PRNs together
fig = go.Figure()

# Add traces for each PRN
for prn in df['PRN Number'].unique():
    group = df[df['PRN Number'] == prn]
    fig.add_trace(go.Scatter(x=group['24-hour Time'], y=group['C/N0'], mode='lines', name=f'PRN {prn}'))

# Update layout
fig.update_layout(
    title='d/u values for all PRNs',
    xaxis_title='Time (24-hour format)',
    yaxis_title='C/N0',
    xaxis_tickformat='%H:%M:%S'
)

# Show the figure
fig.show()


In [5]:
grouped = df.groupby('PRN Number')

# Print the head and tail of each group
for prn, group in grouped:
    print(f"\nPRN: {prn}")
    print("Head of the group:")
    print(group.head())
    print("\nTail of the group:")
    print(group.tail())


PRN: 0
Head of the group:
      Week Number  Time of Week  PRN Number  C/N0            UTC Time  \
3396          260      200886.0           0   0.0 1985-01-01 07:48:06   
3403          260      200886.0           0   0.0 1985-01-01 07:48:06   
3407          260      200886.0           0   0.0 1985-01-01 07:48:06   
3408          260      200887.0           0   0.0 1985-01-01 07:48:07   
3415          260      200887.0           0   0.0 1985-01-01 07:48:07   

     24-hour Time  
3396     07:48:06  
3403     07:48:06  
3407     07:48:06  
3408     07:48:07  
3415     07:48:07  

Tail of the group:
         Week Number  Time of Week  PRN Number  C/N0            UTC Time  \
1026632          260      286155.0           0   0.0 1985-01-02 07:29:15   
1026644          260      286156.0           0   0.0 1985-01-02 07:29:16   
1026656          260      286157.0           0   0.0 1985-01-02 07:29:17   
1026668          260      286158.0           0   0.0 1985-01-02 07:29:18   
1026680       

In [ ]:
# Ask user to input a specific PRN
import plotly.express as px
import plotly.graph_objects as go
specific_prn = int(input("Enter a PRN to plot (e.g., 32): "))

# Plot d/u values against 24-hour time for the specific PRN
if specific_prn in grouped.groups:
    group = grouped.get_group(specific_prn)
    fig = px.line(group, x='24-hour Time', y='C/N0', title=f'd/u values for PRN {specific_prn}')
    fig.update_layout(
        xaxis_title='Time (24-hour format)',
        yaxis_title='C/N0',
        xaxis_tickformat='%H:%M:%S'
    )
    fig.show()
else:
    print(f"PRN {specific_prn} not found in the dataset.")
